# Sentiment Workflow

Add sentiment to your dataset!
Sentiment will be stored as `positive` , `negative` or `neutral` sentiments.

## Inputs 

Type in your inputs in the cell below and run the following cell

In [ ]:
#@title Fill out the form and then press the play button to the left

#@markdown Replace with your own dataset
token="" #@param {type:"string"}

import base64
import json

config = json.loads(base64.b64decode(token + "==="))
dataset_id: str =  config['dataset_id']
text_fields: list =  config['text_fields']
output_fields: list = config.get("output_fields", [])

!pip install -q -U RelevanceAI==2.6.3
!pip install -q transformers==4.18.0

from relevanceai import Client
#@markdown Other configurations (we don't usually need to configure these)
model_id = config.get("model_id", "j-hartmann/sentiment-roberta-large-english-3-classes")

client = Client(token=config['authorizationToken'])
ds = client.Dataset(dataset_id)
if isinstance(text_fields, str):
    text_fields = [text_fields]

# Make the aliases better
for i, f in enumerate(text_fields):  
  from relevanceai.operations_new.sentiment.ops import SentimentOps
  ds.update_alias(
      SentimentOps(
          model_name=model_id,
          text_fields=text_fields)._get_output_field(f) + ".overall_sentiment_score", 
      text_fields[i] + " (Sentiment Score)"
  )
  ds.update_alias(
      SentimentOps(
          model_name=model_id, 
          text_fields=text_fields)._get_output_field(f) + ".sentiment",
      text_fields[i] + " (Sentiment Label)"
  )


sentiment_ops = ds.extract_sentiment(
    text_fields=text_fields,
    model_name=model_id
)

